In [1]:
import concurrent.futures
import pandas as pd
import geopandas as gpd
import mercantile
import h3
import matplotlib.pyplot as plt
import plotly.express as px
from shapely.geometry import Point
from shapely.geometry import shape
from shapely.geometry import box
import multiprocessing as mp

114 m per side (13004 sq meters) at zoom 18

228 m per side (52016 sq meters) at zoom 17

445 m per side (198709 sq meters) at zoom 16

892 m per side (794783 sq meters) at zoom 15


[A typical field is ~200 sq meters](https://www.sciencedirect.com/science/article/pii/S0034425715301851) which would be zoom 17, but that makes the data much harder to work with. Going with zoom 15 for now and will upsample if we think it would make the data viz better.

At zoom 17 we're working with 247626816 tiles. It took 92 minutes 28 seconds.
At zoom 15 we're working with 15477408 tiles and it took about 10 minutes.

In [ ]:
import geopandas as gpd
import mercantile
from shapely.geometry import box
bbox = [-125.0011, 24.8467, -66.9326, 49.9499]
tiles = list(mercantile.tiles(*bbox, zooms=[15]))
polygons = [mercantile.bounds(tile) for tile in tiles]
print("step 1 done")
geometries = [box(bbox[0], bbox[1], bbox[2], bbox[3]) for bbox in polygons]
print("step 2 done")
gdf = gpd.GeoDataFrame(geometry=geometries)
print("step 3 done")
gdf['quadkey'] = [mercantile.quadkey(tile) for tile in tiles]
print("step 4 done")
gdf = gdf.set_crs(epsg=4326)
print("step 5 done")
gdf

In [88]:
# Stand alone function to convert a bounding box to a grid of tiles
import geopandas as gpd
import mercantile
from shapely.geometry import box
def bbox_to_grid (bbox, zoom):
  tiles = list(mercantile.tiles(*bbox, zooms=[zoom]))
  print("Tile List Generated")
  polygons = [mercantile.bounds(tile) for tile in tiles]
  print("Polygons Generated")
  geometries = [box(bbox[0], bbox[1], bbox[2], bbox[3]) for bbox in polygons]
  print("Polygons converted to Shapely Geometries")
  gdf = gpd.GeoDataFrame(geometry=geometries)
  gdf['quadkey'] = [mercantile.quadkey(tile) for tile in tiles]
  print("Quadkeys added to GeoDataFrame")
  gdf = gdf.set_crs(epsg=4326)
  return (gdf)

# test_box = [-125.0011, 24.8467, -66.9326, 49.9499]
# bbox_to_grid(test_box)

This code does spatial joins of states and counties with grid

In [14]:
grid = gpd.read_parquet("../unsynced-data/grid_z15.parquet")
states = gpd.read_file("../unsynced-data/cb_2017_us_state_500k/cb_2017_us_state_500k.shp")
counties = gpd.read_file("../unsynced-data/cb_2017_us_county_500k/cb_2017_us_county_500k.shp")
counties = gpd.read_file("../unsynced-data/cb_2017_us_county_500k/cb_2017_us_county_500k.shp")
counties.columns = [col + '_counties' if col != 'geometry' else col for col in counties.columns]
states = states.to_crs(epsg=4326)
counties = counties.to_crs(epsg=4326)
grid = grid.to_crs(epsg=4326)

In [ ]:
# do a spatial join of gdf and states
grid = gpd.sjoin(grid, states, how="left", op='intersects')
grid = grid.drop(columns=['index_right'])
print ("States join is finished")


In [ ]:
grid = gpd.sjoin(grid, counties, how="left", op='intersects')
print ("Counties join is finished")
grid

Handling names of columns in the grid dataset

In [ ]:
grid = grid.drop(columns=['index_right'])
grid = grid.dropna(subset=['NAME'])

In [ ]:

grid = grid.drop(columns=['STATEFP_counties'])
grid = grid.rename(columns={'COUNTYFP_counties': 'COUNTYFP'})
grid = grid.rename(columns={'AFFGEOID_counties': 'AFFGEOID'})
grid = grid.rename(columns={'GEOID_counties': 'GEOID_COUNTY'})
grid = grid.rename(columns={'NAME_counties': 'NAME_COUNTY'})
grid = grid.rename(columns={'LSAD_counties': 'LSAD_COUNTY'})
grid = grid.rename(columns={'ALAND_counties': 'ALAND_COUNTY'})
grid = grid.rename(columns={'AWATER_counties': 'AWATER_COUNTY'})
grid = grid.rename(columns={'COUNTYNS_counties': 'COUNTYNS'})
grid = grid.rename(columns={'GEOID': 'GEOID_STATE'})
grid = grid.rename(columns={'NAME': 'NAME_STATE'})
grid = grid.rename(columns={'LSAD': 'LSAD_STATE'})
grid = grid.rename(columns={'ALAND': 'ALAND_STATE'})
grid = grid.rename(columns={'AWATER': 'AWATER_STATE'})
grid = grid.rename(columns={'COUNTYNS_counties': 'COUNTYNS'})
grid = grid.reset_index(drop=True)

Final column names

In [89]:
grid.columns

Index(['geometry', 'quadkey', 'STATEFP', 'STATENS', 'GEOID_STATE', 'STUSPS',
       'NAME_STATE', 'LSAD_STATE', 'ALAND_STATE', 'AWATER_STATE', 'COUNTYFP',
       'COUNTYNS', 'GEOID_COUNTY', 'NAME_COUNTY', 'LSAD_COUNTY',
       'ALAND_COUNTY', 'AWATER_COUNTY'],
      dtype='object')

Writing to files

In [70]:
# write to parquet file
grid.to_parquet("../unsynced-data/grid_z15.parquet")

In [73]:
grid.to_file("../unsynced-data/grid_z15.geojson", driver='GeoJSON')

Once the `grid` dataset was compiled, I wrote to a geojson file. I downloaded TIF Cropscape data across the US [from this site](https://nassgeodata.gmu.edu/CropScape/).

I merged the state data from this site using GDAL command: `gdal_merge.py -o outfile.tif in-file1.tif in-file2.tif -n 0`

Once I had a merged tif and grid data in geojson I ran a zonal histogram in QGIS. I would have rather run this with a parquet file instead of geojson, but it wasn't supported in my version of QGIS.

I used the `grid` dataframe as a geojson file and ran a zonal histogram with crop raster data in QGIS.
